# Merit order evaluation
* Create an merit order plot from input data
* Merit order plot is a snapshot situation for one given RES infeed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from pommesevaluation.merit_order import (
    reshape_merit_order_data, plot_merit_order, 
    prepare_market_values_from_results,
    extract_costs_data_for_investment_model,
)
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, 
    aggregate_investment_results,
)

## Define Settings

In [ ]:
# Input and output folders
mode = "dispatch"  # "dispatch", "invest"

input_folder = f'./model_inputs/pommes{mode}/'
results_folder = f'./model_results/pommes{mode}/'
path_plots = "./plots/"

# Settings
simulation_year = 2017
fuel_cost_pathway = 'NZE'
emissions_cost_pathway = 'long-term'
time = f'{simulation_year}-04-30 14:00:00'  #regular situation: f'{simulation_year}-08-05 13:00:00'

# Dispatch model
market_values_from_input = False
eeg_clusters_per_technology = 20
value_exogenous = 50 * 10  # ct/kWh converted to €/MWh
file_addendum = f"{eeg_clusters_per_technology}_clusters"

# Investment model
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "50"
fuel_price_scenario = "NZE"
emissions_pathway = "long-term"
multi_header = False
impose_investment_maxima = False
filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}"
    )
file_extension = ".csv"
rounding_precision = 2

file_name_emission_factors = {
    "dispatch": f"{input_folder}sources_commodity_{simulation_year}.csv",
    "invest": f"{input_folder}sources_commodity.csv",
}

## Read in and prepare information
* Read in
  * costs: fuel, operation, emissions
  * capacities: for conventional and renewables
  * time series: renewable infeed and conventional transformers availability
* Combine into transformers resp. renewables data sets

### Generators - Converters (formerly: transformers)

In [ ]:
if mode == "invest":
    # Read in and preprocess investment and production results from investment model
    investment_results_raw = pd.read_csv(f"{results_folder}{filename}{file_add_on}_investment{file_extension}", index_col=0)
    processed_investment_results = preprocess_raw_results(investment_results_raw)
    processed_investment_results["unit"] = "DE_transformer_" + processed_investment_results["unit"] + "_new_built"
    processed_investment_results = processed_investment_results.set_index("unit")

    # Exogenous transformers
    transformers_exogenous = pd.read_csv(f"{input_folder}transformers_exogenous.csv", index_col=0)
    transformers_exogenous = transformers_exogenous.loc[transformers_exogenous["to_el"] == "DE_bus_el"]
    transformers_exogenous_max_ts = pd.read_csv(f"{input_folder}transformers_exogenous_max_ts.csv", index_col=0)
    transformers_exogenous["capacity"] *= transformers_exogenous_max_ts.loc[f"{time[:4]}-01-01"]
    
    # Invested transformers
    transformers_new_built = pd.read_csv(f"{input_folder}transformers_investment_options.csv", index_col=0)
    transformers_new_built["capacity"] = processed_investment_results.loc[processed_investment_results["year"] == time[:4], "total"]
    
    # Combine
    transformers = pd.concat([transformers_exogenous, transformers_new_built])
    
    # fuel costs
    costs_fuel = extract_costs_data_for_investment_model(
        file_name=f"{input_folder}costs_fuel_{fuel_cost_pathway}_nominal_indexed_ts.csv", 
        frequency=frequency, 
        time=time, 
        col_name="costs_fuel",
    )
    costs_fuel.index.name = "fuel"

    # operation costs
    costs_opex = extract_costs_data_for_investment_model(
        file_name=f"{input_folder}costs_operation_nominal_indexed_ts.csv", 
        frequency=frequency, 
        time=time, 
        col_name="costs_opex",
    )
    costs_opex.index.name = "fuel"

    # emission costs
    costs_emissions = extract_costs_data_for_investment_model(
        file_name=f"{input_folder}costs_emissions_{emissions_cost_pathway}_nominal_indexed_ts.csv", 
        frequency=frequency, 
        time=time, 
        col_name="costs_emissions",
    ).iloc[0, 0].item()

    transformers['carbon_price'] = costs_emissions
    
    del investment_results_raw

In [ ]:
if mode == "dispatch":
    # transformers (i.e. generators)
    transformers = pd.read_csv(input_folder+f'transformers_{simulation_year}.csv')
    transformers = transformers.loc[
        transformers['country'] == 'DE', 
        ['label', 'fuel', 'capacity', 'efficiency_el']
    ]

    # availabilities
    availability_ts = pd.read_csv(input_folder+f'transformers_availability_ts_{simulation_year}.csv', index_col=0)
    availability_factor = availability_ts.at[time, "values"]
    transformers['capacity'] *= availability_factor

    # fuel costs
    costs_fuel = pd.read_csv(
        input_folder+f'costs_fuel_{fuel_cost_pathway}_nominal_{simulation_year}.csv'
    )
    costs_fuel['fuel'] = [_[2] for _ in costs_fuel['label'].str.split('_')]
    costs_fuel = costs_fuel.loc[costs_fuel['label'].str.contains('DE'), ['fuel', f'{simulation_year}']]
    costs_fuel.rename(columns={f'{simulation_year}': 'costs_fuel'}, inplace=True)
    costs_fuel.set_index("fuel", inplace=True)

    # time-dependency for fuel costs
    costs_fuel_ts = pd.read_csv(input_folder+f'costs_fuel_ts_{simulation_year}.csv', index_col=0)
    costs_fuel_factor = costs_fuel_ts.loc[[time]].T
    costs_fuel_factor.reset_index(inplace=True)
    costs_fuel_factor[["country", "source", "fuel"]] = costs_fuel_factor["index"].str.split("_", expand=True)
    costs_fuel_factor = costs_fuel_factor.drop_duplicates(subset="fuel").set_index("fuel")[time]
    costs_fuel["costs_fuel"] = costs_fuel["costs_fuel"] * costs_fuel_factor
    costs_fuel.reset_index()

    # operation costs
    costs_opex = pd.read_csv(input_folder+f'costs_operation_nominal_{simulation_year}.csv')
    costs_opex['fuel'] = [_[2] for _ in costs_opex['label'].str.split('_')]
    costs_opex = costs_opex.loc[costs_opex['label'].str.contains('DE'), ['fuel', f'{simulation_year}']]
    costs_opex.rename(columns={f'{simulation_year}': 'costs_opex'}, inplace=True)

    # emission costs
    costs_emissions = pd.read_csv(input_folder+f'costs_emissions_{emissions_cost_pathway}_nominal_{simulation_year}.csv', index_col=0)
    carbon_price = costs_emissions.at['DE_source_biomass', f'{simulation_year}']
    transformers['carbon_price'] = carbon_price

Same operations for dispatch & invest

In [ ]:
# emission factors
sources_commodity = pd.read_csv(file_name_emission_factors[mode])
sources_commodity['fuel'] = [_[2] for _ in sources_commodity['label'].str.split('_')]
sources_commodity = sources_commodity.loc[sources_commodity['label'].str.contains('DE'), ['fuel', 'emission_factors']]

# Merge costs data and transformers
transformers = transformers.merge(costs_fuel, on='fuel', how='left')
transformers = transformers.merge(costs_opex, on='fuel', how='left')
transformers = transformers.merge(sources_commodity, on='fuel', how='left')
transformers['costs_carbon'] = transformers['emission_factors'] * transformers['carbon_price']

### Renewables

In [ ]:
if mode == "invest":
    if multi_header:
        header = [0, 1]
    else:
        header = 0
    production_results_raw = pd.read_csv(
        f"{results_folder}{filename}{file_add_on}_production{file_extension}", index_col=0, header=header
    ).T
    processed_results = preprocess_raw_results(
        production_results_raw, investments=False, multi_header=multi_header
    ).drop(columns="year").round(rounding_precision)
    aggregated_production_results = aggregate_investment_results(
        processed_results, energy_carriers=["biomass"], by="energy_carrier", investments=False
    ).T

    # RES generation from production results
    res_generation = aggregated_production_results[[
        col for col in aggregated_production_results.columns 
        if "DE_source" in col and not "_shortage" in col
    ] + ["biomass"]]
    
    # Prepare merit order fraction
    sources_res = pd.DataFrame(columns=['label', 'fuel', 'efficiency_el', 'capacity', 'costs_fuel'])
    sources_res["capacity"] = res_generation.loc[time]
    sources_res["label"] = res_generation.columns
    sources_res["fuel"] = [val[-1] for val in sources_res['label'].str.split('_')]
    sources_res["efficiency_el"] = 1
    sources_res["costs_fuel"] = 0
    sources_res.reset_index(drop=True)
    
    # Not required
    transformers_res = pd.DataFrame()
    
    del production_results_raw, processed_results

In [ ]:
if mode == "dispatch":
    # fluctuating renewables capacity factors from time series
    sources_res_ts = pd.read_csv(input_folder+f'sources_renewables_ts_{simulation_year}.csv', index_col=0)
    sources_res_ts = sources_res_ts[sources_res_ts.columns[sources_res_ts.columns.str.contains('DE')]]
    sources_res_ts = sources_res_ts.loc[time]
    sources_res_ts = sources_res_ts.to_frame()
    sources_res_ts.columns = ['capacity_factor']
    
    # res capacities for non-fluctuating renewables
    sources_res = pd.read_csv(
        input_folder+f'sources_renewables_{simulation_year}.csv', 
        usecols=['label', 'capacity', 'country']
    )
    sources_res['fuel'] = [_[2] for _ in sources_res['label'].str.split('_')]
    sources_res = sources_res.merge(sources_res_ts, left_on='label', right_index=True, how='left')
    sources_res['capacity'] *= sources_res['capacity_factor']
    sources_res = sources_res.loc[sources_res['country'] == 'DE', ['fuel', 'capacity']]
    sources_res.replace({'ROR': 'Wasser', 'Klaergas': 'Klärgas'}, inplace=True)
    sources_res['costs_fuel'] = 0
    sources_res['efficiency_el'] = 1
    sources_res['label'] = sources_res['fuel']
    
    # fluctuating renewables (renewable transformers)
    transformers_res = pd.read_csv(
        input_folder+f'transformers_renewables_{file_addendum}_{simulation_year}.csv'
    )

    # renewables operation costs
    costs_res = pd.read_csv(input_folder+f'costs_operation_renewables_{simulation_year}.csv')
    transformers_res = transformers_res.merge(costs_res, on='label', how='left')

    transformers_res = transformers_res.merge(
        sources_res_ts, left_on='from', right_index=True, how='left'
    )
    transformers_res['capacity'] *= transformers_res['capacity_factor']

    transformers_res['fuel'] = [_[1] for _ in transformers_res['label'].str.split('_')]

    # market values renewables
    if market_values_from_input:
        costs_market_values = pd.read_csv(input_folder+f'costs_market_values_{simulation_year}.csv', index_col=0)
    else:
        costs_market_values = pd.read_csv(
            f'{results_folder}dispatch_LP_start-{simulation_year}'
            + f'-01-01_364-days_simple_complete_monthly_market_values_UPDATE.csv', 
            index_col=0
        )
        costs_market_values = prepare_market_values_from_results(costs_market_values, simulation_year)
    costs_market_values = costs_market_values.loc[time].to_frame()
    costs_market_values.columns = ['market_values']
    transformers_res = transformers_res.merge(costs_market_values, right_index=True, left_on='from', how='left')

    # derive costs for renewable transformers
    transformers_res['costs_fuel'] = transformers_res['costs']+transformers_res['market_values']
    exog_trafos = ['DE_solarPV_cluster_exogenous', 'DE_windoffshore_cluster_exogenous', 'DE_windonshore_cluster_exogenous']
    transformers_res.loc[transformers_res['label'].isin(exog_trafos), 'costs_fuel'] = -value_exogenous

    transformers_res = transformers_res[['label', 'fuel', 'efficiency_el', 'capacity', 'costs_fuel']]

In [ ]:
if mode == "dispatch":
    # shortage sources (peaking plants)
    sources_shortage_el_add = pd.read_csv(f"{input_folder}/sources_shortage_el_add_{simulation_year}.csv")
    sources_shortage_el_add.rename(
        columns={
            "shortage_costs": "costs_fuel", 
            "Unnamed: 0": "label",
            "nominal_value": "capacity",
        }, 
        inplace=True
    )
    sources_shortage_el_add["fuel"] = "peaking_units"
    sources_shortage_el_add["efficiency_el"] = 1

else:
    # Not needed
    sources_shortage_el_add = pd.DataFrame()

## Combine different data sets
* Merge all data sets
* Calculate marginal costs
* Sort values by marginal costs and calculate cumulated capacities

In [ ]:
# merge
blocks = pd.concat([transformers, sources_res, transformers_res, sources_shortage_el_add]).fillna(0)

# derive marginal costs
blocks['costs_marginal'] = (
    blocks["costs_opex"] +
    (blocks["costs_fuel"] + blocks["costs_carbon"]) / (blocks['efficiency_el'])
)

# derive cumulated capacity
blocks = blocks.sort_values(by='costs_marginal', ascending=True)
blocks["capacity_cumulated"] = blocks["capacity"].cumsum()

# assign symbolic euro(s) for illustration purposes
blocks.loc[blocks['costs_marginal'] == 0, 'costs_marginal'] = 2

In [ ]:
fuel_dict_DE = {
    'solarPV': 'Solare Strahlungsenergie',
    'windonshore': 'Windenergie an Land',
    'windoffshore': 'Windenergie auf See',
    'uranium': 'Kernenergie',
    'lignite': 'Braunkohle',
    'otherfossil': 'Andere fossile',
    'hardcoal': 'Steinkohle',
    'waste': 'Abfall',
    'mixedfuels': 'Mehrere fossile',
    'biomass': 'Biomasse',
    'biomassEEG': 'Biomasse',
    'landfillgas': 'Deponiegas',
    'geothermal': 'Geothermie',
    'minegas': 'Grubengas',
    'larga': 'Klärgas',
    'natgas': 'Erdgas',
    'oil': 'Heizöl',
    'peaking_units': 'Spitzenlastkraftwerke',
    'hydrogen': 'Wasserstoff',
}
fuel_dict_EN = {
    "biomassEEG": "biomass",
    "Wasser": "water",
}
blocks['fuel'].replace(fuel_dict_EN, inplace=True)

In [ ]:
# Define a color combination
colors_DE = {
    'Solare Strahlungsenergie': '#fcb001',
    'Windenergie auf See': '#0504aa',
    'Windenergie an Land': '#82cafc',
    'Biomasse': '#15b01a',
    'Deponiegas': '#06c2ac',
    'Geothermie': '#ff474c',
    'Wasser': '#c79fef', 
    'Grubengas': '#650021', 
    'Klärgas': '#ad8150', 
    'Kernenergie': '#e50000',
    'Braunkohle': '#7f2b0a',
    'Andere fossile': '#d8dcd6',
    'Steinkohle': '#000000', 
    'Abfall': '#c04e01',
    'Mehrere fossile': '#a57e52',
    'Erdgas': '#929591', 
    'Heizöl': '#aaa662',
    'Spitzenlastkraftwerke': '#999999',
    'Wasserstoff': '#6fa8dc',
}
colors_EN = {
    'solarPV': '#fcb001',
    'windoffshore': '#0504aa',
    'windonshore': '#82cafc',
    'biomass': '#15b01a',
    'landfillgas': '#06c2ac',
    'geothermal': '#ff474c',
    'water': '#c79fef', 
    'minegas': '#650021', 
    'larga': '#ad8150', 
    'uranium': '#e50000',
    'lignite': '#7f2b0a',
    'otherfossil': '#d8dcd6',
    'hardcoal': '#000000', 
    'waste': '#c04e01',
    'mixedfuels': '#a57e52',
    'natgas': '#929591', 
    'oil': '#aaa662',
    'peaking_units': '#999999',
    'hydrogen': '#6fa8dc',
    "ROR": "blue",
}

## Create actual merit order plot
* Reshape data set
* plot using steps

In [ ]:
# Reshape data set
merit_order = reshape_merit_order_data(blocks)

plt.rcParams.update({'font.size': 12})

# Create actual plot
plot_merit_order(
    merit_order, 
    colors_EN,
    ylim=(-90, 400), 
    interval=20, 
    set_ylim=True, 
    height=10,
    save=True,
    path_plots=path_plots,
    file_name=f"merit_oder_for_{simulation_year}{file_addendum}",
)

## Show only negative portion of merit order
* Exclude conventionals and every unit with marginal costs above 0
* Include some renaming to create nice plots for a paper

In [ ]:
if mode == "dispatch":
    cutted_merit_order = reshape_merit_order_data(blocks.loc[blocks["costs_marginal"] <= 2])

    tidy_fuel_names = {
        "solarPV": "solar photovoltaics",
        "windonshore": "wind onshore",
        "windoffshore": "wind offshore",
        "minegas": "gaseous RES",
        "landfillgas": "gaseous RES",
        "larga": "gaseous RES",
        "geothermal": "geothermal energy",
        "water": "run of river",
    }

    colors_EN_tidy = {
        'solar photovoltaics': '#fcb001',
        'wind offshore': '#0504aa',
        'wind onshore': '#82cafc',
        'biomass': '#15b01a',
        'gaseous RES': '#ad8150',
        'geothermal energy': '#ff474c',
        'run of river': '#c79fef', 
    }

    cutted_merit_order = cutted_merit_order.replace(tidy_fuel_names)

    # Create actual plot
    plot_merit_order(
        cutted_merit_order, 
        colors_EN_tidy,
        ylim=(-520, 20), 
        interval=20, 
        set_ylim=True, 
        height=10,
        save=True,
        path_plots=path_plots,
        file_name=f"cutted_merit_oder_for_{simulation_year}{file_addendum}",
        place_legend_below=True
    )